##Import Library

In [ ]:
import pandas as pd
import numpy as np
import os
from glob import glob
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.optimizers import adam_v2, adagrad_v2, adadelta_v2, sgd_experimental, rmsprop_v2
# from keras.optimizers import Adam, Adagrad, Adadelta, SGD, RMSprop
from keras.layers import Dense, Dropout, Activation, Conv2D
from keras.layers import Flatten, MaxPool2D, MaxPooling2D, BatchNormalization
%matplotlib inline

##Processing Dataset

In [ ]:
path = "/content/drive/MyDrive/Dataset_BUSI_with_GT/"
imagepatches = glob("/content/drive/MyDrive/Dataset_BUSI_with_GT/**/*).png",recursive=True)
# for filename in imagepatches[0:10]:
#   print(filename)

In [ ]:
import random

class0 = []
class1 = []
class2 = []

for filename in imagepatches:
  if filename.find("normal") != -1:
    class0.append(filename)
  elif filename.find("malignant") != -1:
    class1.append(filename)
  else:
    class2.append(filename)

#randomizing the values
sample_class0 = random.sample(class0,len(class0))
sample_class1 = random.sample(class1,len(class1))
sample_class2 = random.sample(class2,len(class2))

In [ ]:
#for image resoltuion 50
from matplotlib.image import imread
import cv2

size = 128
def get_image_arrays(data,label,size=(50,50)):
  img_arrays = []
  for i in data:
    if i.endswith(".png"):
      img = cv2.imread(i,cv2.IMREAD_COLOR)
      img_sized = cv2.resize(img, size, interpolation=cv2.INTER_LINEAR)
      img_arrays.append([img_sized.flatten(),label])
  return img_arrays

# class0_array_50 = get_image_arrays(sample_class0,0,(size,size))
# class1_array_50 = get_image_arrays(sample_class1,1,(size,size))
# class2_array_50 = get_image_arrays(sample_class2,2,(size,size))

class0_array_128 = get_image_arrays(sample_class0,0,(size,size))
class1_array_128 = get_image_arrays(sample_class1,1,(size,size))
class2_array_128 = get_image_arrays(sample_class2,2,(size,size))

In [ ]:
import random
# combined_data = np.concatenate((class0_array_50,class1_array_50))
# combined_data = np.concatenate((combined_data,class2_array_50))

combined_data = np.concatenate((class0_array_128,class1_array_128))
combined_data = np.concatenate((combined_data,class2_array_128))

random.seed(42)
random.shuffle(combined_data)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
def combined_data_fn(combined_data, shape = (-1,128,128,3)):
  X = []
  y = []
  for features,label in combined_data:
    X.append(features)
    y.append(label)
  flat_data=np.array(X)
  target=np.array(y)
  df=pd.DataFrame(flat_data) #dataframe
  df['Target']=target
  x=df.iloc[:,:-1] #input data 
  y=df.iloc[:,-1] #output data
  print(x.shape)
  return x,y

x, y = combined_data_fn(combined_data)

(780, 49152)


## Training

In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

def make_model(C = 1,gamma = 'scale',kernel = 'rbf'):
  param_grid={'C':[0.1,1,10,100],'gamma':[0.0001,0.001,0.1,1],'kernel':['rbf','poly']}
  svc=svm.SVC(probability=True)
  model=GridSearchCV(svc,param_grid)
  clf= svm.SVC(C = C, gamma = gamma, kernel='rbf')
  return clf

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

##Dataset Split into train, test, validation set
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.10,random_state=77,stratify=y)
# x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.05,random_state=77,stratify=y_train)
print('Dataset Splitted')



clf = make_model(C = 10,gamma = 0.5,kernel = 'rbf')
print("Training Model")
clf.fit(x_train,y_train)
print('The Model is trained well with the given images')
# model.best_params_ contains the best parameters obtained from GridSearchCV

y_pred=clf.predict(x_test)
print(f"The model is {accuracy_score(y_pred,y_test)*100}% accurate")

Dataset Splitted
Training Model
The Model is trained well with the given images
The model is 82.05128205128204% accurate
